In [1]:
import pandas as pd
import networkx as nx
from sodapy import Socrata
import math
import random
import matplotlib.pyplot as plt
import numpy as np
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import copy 

#to supress some annoying warnings
# import warnings; warnings.simplefilter('ignore')

In [2]:
# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.cityofnewyork.us", None)

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("2yzn-sicd", limit=20000)

# Convert to pandas DataFrame
data = pd.DataFrame.from_records(results)

In [3]:
data

,dropoff_datetime,dropoff_latitude,dropoff_longitude,extra,fare_amount,mta_tax,passenger_count,payment_type,pickup_datetime,pickup_latitude,pickup_longitude,rate_code,store_and_fwd_flag,tip_amount,tolls_amount,total_amount,trip_distance,vendor_id
0,2015-02-04T13:06:03.000,40.7449951171875,-74.002365112304687,0,18,0.5,5,1,2015-02-04T12:38:47.000,40.767932891845703,-73.9622802734375,1,N,3.76,0,22.56,3.46,2
1,2015-06-06T11:01:59.000,40.772018432617187,-73.955841064453125,0,7.5,0.5,1,1,2015-06-06T10:54:25.000,40.765346527099609,-73.975196838378906,1,N,1.65,0,9.95,1.6,1
2,2015-04-20T18:31:00.000,40.704860687255859,-74.017387390136719,1,3.5,0.5,5,1,2015-04-20T18:29:06.000,40.708278656005859,-74.01763916015625,1,N,1.5,0,6.8,0.35,2
3,2015-05-09T23:26:40.000,40.763805389404297,-73.925048828125,0.5,24,0.5,1,2,2015-05-09T23:00:54.000,40.727981567382813,-73.982185363769531,1,N,0,0,25.3,6.6,1
4,2015-03-26T20:27:57.000,40.757457733154297,-73.963676452636719,0.5,17,0.5,1,2,2015-03-26T20:14:30.000,40.706417083740234,-74.005531311035156,1,N,0,0,18.3,5.1,1
5,2015-06-12T19:40:05.000,40.764270782470703,-73.995704650878906,1,6,0.5,2,1,2015-06-12T19:34:16.000,40.754261016845703,-73.995773315429688,1,N,1.56,0,9.36,1.06,2
6,2015-04-17T19:28:10.000,40.735210418701172,-74.000869750976563,1,7,0.5,1,1,2015-04-17T19:19:29.000,40.726581573486328,-73.994110107421875,1,N,1.76,0,10.56,0.98,2
7,2015-05-21T17:46:20.000,40.643581390380859,-73.783897399902344,0,52,0.5,2,1,2015-05-21T16:12:02.000,40.746078491210937,-73.990531921386719,2,N,2,5.54,60.34,16,2
8,2015-01-31T07:48:46.000,40.748668670654297,-74.008171081542969,0,8,0.5,2,1,2015-01-31T07:39:56.000,40.741783142089844,-73.989753723144531,1,N,1,0,9.8,1.3,1
9,2015-01-23T23:27:30.000,40.714504241943359,-73.941871643066406,0.5,4,0.5,3,2,2015-01-23T23:24:38.000,40.707309722900391,-73.939704895019531,1,N,0,0,5.3,0.5,1


In [4]:
geolocator = Nominatim(user_agent="my-application")
reverse = RateLimiter(geolocator.reverse, min_delay_seconds=2)

In [5]:
def to_zip(lat,lon):
    cd = copy.deepcopy(lat)
    cd += ","
    cd += copy.deepcopy(lon)
#     print("query: ", cd)
    loc = reverse(cd)
    try:
        res = int(loc.raw['address']['postcode'])
        return res 
    except:
        return 0

In [ ]:
dropoff_zip = []
pickup_zip = []
for i in range(data.shape[0]):
    d_lat = data['dropoff_latitude'][i]
    d_lon = data['dropoff_longitude'][i]
    
    p_lat = data['pickup_latitude'][i]
    p_lon = data['pickup_longitude'][i]
    
    dropoff_zip.append(to_zip(d_lat, d_lon))
    pickup_zip.append(to_zip(p_lat, p_lon))
    
data.insert(loc=len(data.columns),column='dropoff_zip', value=dropoff_zip)
data.insert(loc=len(data.columns),column='pickup_zip', value=pickup_zip)
data

In [ ]:
data.to_pickle("data")

In [ ]:
# pos_2d = [(x % grid_size, math.floor(x/grid_size), ) for x in range(grid_size*grid_size)]
# pos_mapping = dict(zip(range(grid_size*grid_size), pos_2d))
# pos_mapping
# nx.draw(G, pos_mapping)

In [ ]:
# create network graph and initialize nodes based on grid size 
G = nx.DiGraph(); 
for ride in range(data.shape[0]):
    src = data['pickup_zip'][ride]
    dest = data['dropoff_zip'][ride]
    G.add_edge(src, dest)
    


In [ ]:
# G.in_degree[]

In [ ]:
in_degree = np.zeros(100)
out_degree = np.zeros(100)

for k in G.nodes:
    in_num = G.in_degree[k]
    out_num = G.out_degree[k]
    in_degree[in_num] += 1
    out_degree[out_num] += 1
print(np.sum(in_degree))
print(np.sum(out_degree))
out_degree

In [ ]:
in_degree